# Project 5 - Vehicle Detection
Course: Udacity Self Driving Car Nanodegree Term 1

Author: Roman Stanchak (rstanchak@gmail.com)

Date: May 5, 2017

The goals / steps of this project are the following:

* Perform a Histogram of Oriented Gradients (HOG) feature extraction on a labeled training set of images and train a classifier Linear SVM classifier
* Optionally, you can also apply a color transform and append binned color features, as well as histograms of color, to your HOG feature vector. 
* Note: for those first two steps don't forget to normalize your features and randomize a selection for training and testing.
* Implement a sliding-window technique and use your trained classifier to search for vehicles in images.
* Run your pipeline on a video stream (start with the test_video.mp4 and later implement on full project_video.mp4) and create a heat map of recurring detections frame by frame to reject outliers and follow detected vehicles.
* Estimate a bounding box for vehicles detected.

[//]: # (Image References)

## [Rubric](https://review.udacity.com/#!/rubrics/513/view) Points
### Here I will consider the rubric points individually and describe how I addressed each point in my implementation.  

---
### Writeup / README

#### 1. Provide a Writeup / README that includes all the rubric points and how you addressed each one.  You can submit your writeup as markdown or pdf.  [Here](https://github.com/udacity/CarND-Vehicle-Detection/blob/master/writeup_template.md) is a template writeup for this project you can use as a guide and a starting point.  

You're reading it!


[image1]: ./output_images/test1.jpg
[image2]: ./output_images/test2.jpg
[image3]: ./output_images/test3.jpg
[image4]: ./output_images/test4.jpg
[image5]: ./output_images/test5.jpg
[image6]: ./output_images/test6.jpg
[video1]: ./output_videos/project_video.mp4
### Histogram of Oriented Gradients (HOG)
#### 1. Explain how (and identify where in your code) you extracted HOG features from the training images.

To extract HOG features from the training images, I started with the example code from the lecture lab utilizing the skimage.feature.hog function, and wrapped this in a HogFeatureExtractor class (in extract_features.py). 
    The HogFeatureExtractor class compartmentalizes the image color conversion, HoG computation, and feature vector creation of ROI windows.  The HoG parameters are encoded in a JSON file (env.json) which is given as an argument to a command-line script (m1_extract_all.py).  This script reads all the images in a directory, computes the HoG features and saves the result to a set of intermediate files.

#### 2. Explain how you settled on your final choice of HOG parameters.

I used LUV colorspace as this provided the improved accuracy on the validation set in the lecture lab.  I used the default HoG parameters as they seemed to be based on the optimal values determined in the academic literature.

#### 3. Describe how (and identify where in your code) you trained a classifier using your selected HOG features (and color features if you used them).

I trained a "C-support" SVM using the sklearn.svm.SVC class.  The training is done in a command-line script (m2_train.py) which reads the feature vectors from the file system, segments the training data into train and validation sets (30% split), fits the classifier, and prints out a brief summary report using sklearn.metrics.classification_report

### Sliding Window Search

#### 1. Describe how (and identify where in your code) you implemented a sliding window search.  How did you decide what scales to search and how much to overlap windows?

To implement sliding window search, I started with the slide_window function from the lecture lab and wrapped this, along with the vehicle detection pipeline in a VehicleDetector class (in vehicle_detection2.py).  I used a gaussian pyramid to search the image at 3 scales to detect vehicles of different sizes.

<img src="examples/search_windows.jpg" width="640"/>

The green grid shows the overlapping search windows at 1:1 scale, red at 1:2 scale and blue at 1:4 scale.

The scales and overlap windows were determined emprically to balance performance and computation time on the test images. 

#### 2. Show some examples of test images to demonstrate how your pipeline is working.  What did you do to optimize the performance of your classifier?

Three test images are shown below to demonstrate the final output of the pipeline.

| Desc | Image |
|---|---|
| Two cars | <img src="output_images/test1.jpg" width="640"/> | 
| No cars | <img src="output_images/test2.jpg" width="640"/> |
| Distant car | <img src="output_images/test3.jpg" width="640"/> |

To optimize the performance of the classifier, I experimented with various classifiers, adjusting the SVM C parameter, and class weights.  I did not come to any meaningful conclusions.

---

### Video Implementation

#### 1. Provide a link to your final video output.  Your pipeline should perform reasonably well on the entire project video (somewhat wobbly or unstable bounding boxes are ok as long as you are identifying the vehicles most of the time with minimal false positives.)

[Here is a link to my video](output_videos/project_video.mp4)


#### 2. Describe how (and identify where in your code) you implemented some kind of filter for false positives and some method for combining overlapping bounding boxes.

I filtered false positives and combined bounding boxes by implementing a spatial heatmap with a temporal decay of 0.75 (vehicle_detection2.py), and a bounding box size penalty proportionate to the scale.  The heatmap is thresholded and labeled using the example code from the lecture (i.e. using scipy.ndimage.measurements.label).

---

### Discussion

#### 1. Briefly discuss any problems / issues you faced in your implementation of this project.  Where will your pipeline likely fail?  What could you do to make it more robust?

I had a lot of technical issues with this project - NaN's in the HoG features, stupid coding errors, crashes in Jupyter notebook, slow laptop, etc - these wasted alot of time and distracted from the actual project. 

The video pipeline fails to detect vehicles in much of the project video and produces too many false positives.

In addition, vehicle detection is extremely slow, about 3.5s per frame.